# DS-practice[3] Database Design & Build

[학습목표] <b><span style="color:blue"> 데이터베이스를 설계하고 구축할 수 있다. </span></b>

- <b>Database</b> building
    - ACCESS DB
    - INSERT DB
    - SELECT DB

1) <b>ACCESS DB</b>

In [6]:
SERVER_IP = "202.30.31.17"
DB_NAME = "db201620740"

In [2]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


--DB connection을 여는 코드--

In [7]:
import logging
import pymysql

import socket

def _CheckIP():
    return socket.gethostbyname(socket.gethostname()) == SERVER_IP

def _getHostName():    
    if _CheckIP():
        return "localhost"
    else:
        return SERVER_IP
        
def connectDb(user, password):
    host = _getHostName()

    try:
        return pymysql.connect(host = host, user = user, password = password, db = DB_NAME, charset="utf8")
    except:
        logging.error("Check DB info")
        logging.error("user : " + user + ", password : " + password)        

In [4]:
conn=connectDb("std201620740","201620740")
cur=conn.cursor()

2) <b>INSERT DB</b>

In [6]:
import pandas as pd

data = pd.read_csv("C:/Users/wnsdu/OneDrive/바탕 화면/센서빅데이터 처리/practice3/dbACC.csv")
tupleData = [tuple(x) for x in data.values]

In [7]:
tupleData

[(0.0, -0.2087, -0.4806, 0.377),
 (1.0, -0.2104, -0.4819, 0.3759),
 (2.0, -0.2103, -0.4822, 0.3798),
 (3.0, -0.2119, -0.4819, 0.3806),
 (4.0, -0.2148, -0.4812, 0.3823),
 (5.0, -0.2156, -0.4812, 0.3823),
 (6.0, -0.2185, -0.4817, 0.3839),
 (7.0, -0.2201, -0.4805, 0.3848),
 (8.0, -0.2209, -0.4796, 0.3862),
 (9.0, -0.2227, -0.4794, 0.3859),
 (10.0, -0.2253, -0.4786, 0.3881),
 (11.0, -0.2253, -0.4792, 0.3879),
 (12.0, -0.2278, -0.4785, 0.3871),
 (13.0, -0.2302, -0.4767, 0.387),
 (14.0, -0.2312, -0.4758, 0.3872),
 (15.0, -0.2322, -0.4742, 0.3883),
 (16.0, -0.2339, -0.4727, 0.387),
 (17.0, -0.2349, -0.4722, 0.3899),
 (18.0, -0.2363, -0.4722, 0.3936),
 (19.0, -0.2373, -0.4724, 0.3955),
 (20.0, -0.2363, -0.4733, 0.3955),
 (21.0, -0.2373, -0.4749, 0.3945),
 (22.0, -0.2374, -0.4745, 0.3987),
 (23.0, -0.2383, -0.4731, 0.4019),
 (24.0, -0.24, -0.4714, 0.4059),
 (25.0, -0.24100000000000002, -0.4692, 0.405),
 (26.0, -0.2393, -0.4705, 0.4037),
 (27.0, -0.2393, -0.4691, 0.4045),
 (28.0, -0.2397, -0.469

In [9]:
query="insert into accelerator (nSeq, port4_chn1, port6_chn1, port7_chn1) values(%s, %s, %s, %s)"
#accelerator라는 table에 값들을 저장하기 위해 우선적으로 이름을 저장함. data들은 한번에 저장할 예정임으로 %s로 나타냄
cur.executemany(query,tupleData)
#한번에 데이터 많이 저장할 때 cur.executemany를 이용
conn.commit()
#conn으로 데이터 베이스에 연결하였기에 커밋할 때 이를 이용

In [37]:
for i in range(100):
    sql="insert into accelerator (nSeq, port4_chn1, port6_chn1, port7_chn1) values("+ str(data['nSeq'][i]) + ", " + str(data['PORT4_CHN1'][i]) + ", " + str(data['PORT6_CHN1'][i]) + ", " + str(data['PORT7_CHN1'][i])+")" 
    cur.execute(sql)
    conn.commit
#한줄씩 tuple을 db에 저장

In [9]:
conn.close()

3) <b>SELECT DB</b>

In [4]:
conn = connectDb("std201620740", "201620740")
cur = conn.cursor()

query = "SELECT * FROM accelerator WHERE data_idx BETWEEN 0 AND 100"

cur.execute(query)
result = cur.fetchall() #cur의 값을 받아와 result에 저장
print(result[3:5])

()


In [8]:
conn = connectDb("std201620740", "201620740")
cur = conn.cursor()
query = "SELECT AVG(PORT4_CHN1), MAX(PORT6_CHN1),  AVG(PORT7_CHN1) FROM accelerator WHERE data_idx BETWEEN 0 AND 100"

cur.execute(query)
result = cur.fetchall() #cur의 값을 받아와 result에 저장
print(result)

((-0.2089339992403984, -0.4651, 0.40765299886465073),)


In [47]:
query = "SELECT SUM(PORT4_CHN1), SUM(PORT6_CHN1),  AVG(PORT7_CHN1) FROM accelerator WHERE data_idx BETWEEN 0 AND 100"

cur.execute(query)
result = cur.fetchall() #cur의 값을 받아와 result에 저장
print(result)

((-20.89339992403984, -47.703800082206726, 0.40765299886465073),)


In [48]:
query = "SELECT COUNT(data_idx) FROM accelerator WHERE PORT4_CHN1 < 0"

cur.execute(query)
result = cur.fetchall() #cur의 값을 받아와 result에 저장
print(result)

((17962,),)


파이썬에서보다 DB에서 직접 쿼리에 SELECT를 넣어 실행하면, 특히 key값을 주어 실행하면 속도가 굉장히 빠르다.
따라서 빅데이터에서 DB를 사용할 수 밖에 없다.